In [1]:
import pandas as pd


In [12]:
CH_PATH = r"C:\Users\swl00\IFPRI Dropbox\Weilun Shi\Google fund\Analysis\1.Source Data\CH_final_v12102025_with_zscores.csv"
IPC_PATH = r"C:\Users\swl00\IFPRI Dropbox\Weilun Shi\Google fund\Analysis\1.Source Data\IPC_2017_2025_final_v06152025_with_zscores.csv"
popdensity_CH = r"C:\Users\swl00\IFPRI Dropbox\Weilun Shi\Google fund\Analysis\1.Source Data\Populationdensity\ch_population_density_completed_batched.csv"

In [13]:
#read CH_DATA amd popdensity data
ch_df = pd.read_csv(CH_PATH)
popdensity_df = pd.read_csv(popdensity_CH)

In [14]:
# merge on title and ISO3
merged_ch_df = pd.merge(ch_df, popdensity_df[['ISO3', 'title', 'popdensity']], on=['ISO3', 'title'], how='left')

In [15]:
# drop duplicates in lat lon year month
merged_ch_df = merged_ch_df.drop_duplicates(subset=['lat', 'lon', 'year', 'month'])

In [45]:
# read IPC data
ipc_df = pd.read_csv(IPC_PATH)


In [46]:

# see the difference of columns between two dataframes
ch_columns = set(merged_ch_df.columns)
ipc_columns = set(ipc_df.columns)
columns_in_ch_not_in_ipc = ch_columns - ipc_columns
columns_in_ipc_not_in_ch =  ipc_columns - ch_columns
print(columns_in_ch_not_in_ipc)
print(columns_in_ipc_not_in_ch)
# reorder In_CH_not_in_IPC from a-z, do the same for In_IPC_not_in_CH
columns_in_ch_not_in_ipc = sorted(list(columns_in_ch_not_in_ipc))
columns_in_ipc_not_in_ch = sorted(list(columns_in_ipc_not_in_ch))
# convert to dataframe, force to have same length by filling with None
diff_df = pd.DataFrame({
    'In_CH_not_in_IPC': list(columns_in_ch_not_in_ipc) + [None]*(len(columns_in_ipc_not_in_ch) - len(columns_in_ch_not_in_ipc)),
    'In_IPC_not_in_CH': list(columns_in_ipc_not_in_ch) + [None]*(len(columns_in_ch_not_in_ipc) - len(columns_in_ipc_not_in_ch))
})

{'AEZ_7000', 'AEZ_38000', 'AEZ_32000', 'GPP_std_l12', 'country_code_3', 'nightlight_std', 'popdensity', 'GPP_mean_v12', 'area_id', 'sg_cfvo_5-15cm', 'nightlight_mean', 'nightlight_mean_l12', 'EVI_std_v12', 'GPP_mean_m12', 'EVI_std_l12', 'phase4_population', 'phase3_population', 'lon_fixed', 'ISO2', 'EVI_std', 'GPP_std_v12', 'nightlight_mean_m12', 'EVI_std_m12', 'phase2_population', 'sg_phh2o_5-15cm', 'nightlight_std_m12', 'GPP_std_m12', 'lat_fixed', 'nightlight_std_l12', 'GPP_std', 'phase1_population', 'sg_soc_5-15cm', 'phase3_worse_population', 'title', 'EVI_mean_v12', 'nightlight_std_v12', 'phase5_population', 'AEZ_34000', 'EVI_mean', 'EVI_mean_m12', 'sg_nitrogen_5-15cm', 'nightlight_mean_v12', 'AEZ_42000', 'GPP_mean', 'EVI_mean_l12', 'GPP_mean_l12', 'AEZ_36000', 'sg_cec_5-15cm'}
{'EVI_stdDev', 'phase2_percent', 'nightlight_sd', 'gpp_mean_l12', 'aez_groupid_43000', 'nightlight_sd_v12', 'aez_groupid_4000', 'gpp_sd_v12', 'gpp_mean_m12', 'phase4_percent', 'EVI_l12', 'aez_groupid_17000',

In [47]:
# rename IPC columns, EVI o EVI_mean, EVI_l12 to EVI_mean_l12, EVI_m12 to EVI_mean_m12, EVI_stdDev to EVI_std, EVI_stdDev_l12 to EVI_std_l12, EVI_stdDev_m12 to EVI_std_m12, EVI_stdDev_v12 to EVI_std_v12, aez_groupid_* to AEZ_*
ipc_df = ipc_df.rename(columns={
    'EVI': 'EVI_mean',
    'EVI_l12': 'EVI_mean_l12',
    'EVI_m12': 'EVI_mean_m12',
    'EVI_stdDev': 'EVI_std',
    'EVI_stdDev_l12': 'EVI_std_l12',
    'EVI_stdDev_m12': 'EVI_std_m12',
    'EVI_stdDev_v12': 'EVI_std_v12',
})

# search aez pattern
aez_columns = [col for col in ipc_df.columns if col.startswith('aez_groupid_')]
for col in aez_columns:
    new_col_name = col.replace('aez_groupid_', 'AEZ_')
    ipc_df = ipc_df.rename(columns={col: new_col_name})
    
# rename cec_5-15cm_mean to sg_cec_5_15cm_mean, cfvo_5-15cm_mean to sg_cfvo_5_15cm_mean, nitrogen_5-15cm_mean to sg_nitrogen_5_15cm_mean, phh2o_5-15cm_mean to sg_phh2o_5_15cm_mean, soc_5-15cm_mean to sg_soc_5_15cm_mean
ipc_df = ipc_df.rename(columns={
    'cec_5-15cm_mean': 'sg_cec_5-15cm',
    'cfvo_5-15cm_mean': 'sg_cfvo_5-15cm',
    'nitrogen_5-15cm_mean': 'sg_nitrogen_5-15cm',
    'phh2o_5-15cm_mean': 'sg_phh2o_5-15cm',
    'soc_5-15cm_mean': 'sg_soc_5-15cm',
})

In [48]:
#in ipc_df, drop Unnamed columns
ipc_df = ipc_df.loc[:, ~ipc_df.columns.str.contains('^Unnamed')]

# rename EVI_v12 to EVI_mean_v12
ipc_df = ipc_df.rename(columns={
    'EVI_v12': 'EVI_mean_v12',
})

# rename columns started with gpp_ to GPP_
gpp_columns = [col for col in ipc_df.columns if col.startswith('gpp_')]
for col in gpp_columns:
    new_col_name = col.replace('gpp_', 'GPP_')
    ipc_df = ipc_df.rename(columns={col: new_col_name})
    
#rename columns nightlight to nightlight_mean, nightlight_l12 to nightlight_mean_l12, nightlight_m12 to nightlight_mean_m12, nightlight_sd to nightlight_std, nightlight_sd_l12 to nightlight_std_l12, nightlight_sd_m12 to nightlight_std_m12, nightlight_sd_v12 to nightlight_std_v12, nightlight_v12 to nightlight_mean_v12
ipc_df = ipc_df.rename(columns={
    'nightlight': 'nightlight_mean',
    'nightlight_l12': 'nightlight_mean_l12',
    'nightlight_m12': 'nightlight_mean_m12',
    'nightlight_sd': 'nightlight_std',
    'nightlight_sd_l12': 'nightlight_std_l12',
    'nightlight_sd_m12': 'nightlight_std_m12',
    'nightlight_sd_v12': 'nightlight_std_v12',
    'nightlight_v12': 'nightlight_mean_v12',
})

# drop row_id
ipc_df = ipc_df.drop(columns=['row_id'])

In [49]:
# rename GPP_sd to GPP_std, GPP_sd_l12 to GPP_std_l12, GPP_sd_m12 to GPP_std_m12, GPP_sd_v12 to GPP_std_v12
ipc_df = ipc_df.rename(columns={
    'GPP_sd': 'GPP_std',
    'GPP_sd_l12': 'GPP_std_l12',
    'GPP_sd_m12': 'GPP_std_m12',
    'GPP_sd_v12': 'GPP_std_v12',
})

In [50]:

# see the difference of columns between two dataframes
ch_columns = set(merged_ch_df.columns)
ipc_columns = set(ipc_df.columns)
columns_in_ch_not_in_ipc = ch_columns - ipc_columns
columns_in_ipc_not_in_ch =  ipc_columns - ch_columns
print(columns_in_ch_not_in_ipc)
print(columns_in_ipc_not_in_ch)
# reorder In_CH_not_in_IPC from a-z, do the same for In_IPC_not_in_CH
columns_in_ch_not_in_ipc = sorted(list(columns_in_ch_not_in_ipc))
columns_in_ipc_not_in_ch = sorted(list(columns_in_ipc_not_in_ch))
# convert to dataframe, force to have same length by filling with None
diff_df = pd.DataFrame({
    'In_CH_not_in_IPC': list(columns_in_ch_not_in_ipc) + [None]*(len(columns_in_ipc_not_in_ch) - len(columns_in_ch_not_in_ipc)),
    'In_IPC_not_in_CH': list(columns_in_ipc_not_in_ch) + [None]*(len(columns_in_ch_not_in_ipc) - len(columns_in_ipc_not_in_ch))
})

{'ISO2', 'phase2_population', 'lat_fixed', 'phase4_population', 'country_code_3', 'popdensity', 'area_id', 'AEZ_42000', 'phase3_population', 'phase1_population', 'lon_fixed', 'phase3_worse_population', 'title', 'phase5_population'}
{'phase2_percent', 'AEZ_20000', 'AEZ_30000', 'phase4_percent', 'AEZ_28000', 'AEZ_40000', 'AEZ_19000', 'AEZ_9000', 'AEZ_17000', 'AEZ_10000', 'phase3_percent', 'AEZ_4000', 'phase1_percent', 'AEZ_33000', 'AEZ_25000', 'AEZ_31000', 'AEZ_12000', 'AEZ_43000', 'AEZ_35000', 'phase5_percent'}


In [51]:
# in CH dataset, drop ISO2
merged_ch_df = merged_ch_df.drop(columns=['ISO2'])

# rename area_id to admin_code, but add 100000 to area_id first
merged_ch_df['area_id'] = merged_ch_df['area_id'] + 100000
merged_ch_df = merged_ch_df.rename(columns={
    'area_id': 'admin_code'
})

# drop country_code_3
merged_ch_df = merged_ch_df.drop(columns=['country_code_3'])

# drop lat_fixed, lon_fixed
merged_ch_df = merged_ch_df.drop(columns=['lat_fixed', 'lon_fixed'])

#calculate phase1_percent = phase1_population/estimated_population *100
merged_ch_df['phase1_percent'] = (merged_ch_df['phase1_population'] / merged_ch_df['estimated_population'])

#same for phase2_percent, phase3_percent, phase4_percent, phase5_percent
merged_ch_df['phase2_percent'] = (merged_ch_df['phase2_population'] / merged_ch_df['estimated_population'])
merged_ch_df['phase3_percent'] = (merged_ch_df['phase3_population'] / merged_ch_df['estimated_population'])
merged_ch_df['phase4_percent'] = (merged_ch_df['phase4_population'] / merged_ch_df['estimated_population'])
merged_ch_df['phase5_percent'] = (merged_ch_df['phase5_population'] / merged_ch_df['estimated_population'])

#drop phase1_population, phase2_population, phase3_population, phase4_population, phase5_population
merged_ch_df = merged_ch_df.drop(columns=['phase1_population', 'phase2_population', 'phase3_population', 'phase4_population', 'phase5_population'])
# drop phase3_worse_population
merged_ch_df = merged_ch_df.drop(columns=['phase3_worse_population'])



In [52]:
# extract lat, lon, title
ch_lat_lon_title = merged_ch_df[['lat', 'lon', 'title']].drop_duplicates()

# save to csv
ch_lat_lon_title.to_csv(r"C:\Users\swl00\IFPRI Dropbox\Weilun Shi\Google fund\Analysis\1.Source Data\CH_lat_lon_title.csv", index=False)

In [53]:
#merged_ch_df drop title
merged_ch_df = merged_ch_df.drop(columns=['title'])

In [54]:

# see the difference of columns between two dataframes
ch_columns = set(merged_ch_df.columns)
ipc_columns = set(ipc_df.columns)
columns_in_ch_not_in_ipc = ch_columns - ipc_columns
columns_in_ipc_not_in_ch =  ipc_columns - ch_columns
print(columns_in_ch_not_in_ipc)
print(columns_in_ipc_not_in_ch)
# reorder In_CH_not_in_IPC from a-z, do the same for In_IPC_not_in_CH
columns_in_ch_not_in_ipc = sorted(list(columns_in_ch_not_in_ipc))
columns_in_ipc_not_in_ch = sorted(list(columns_in_ipc_not_in_ch))
# convert to dataframe, force to have same length by filling with None
diff_df = pd.DataFrame({
    'In_CH_not_in_IPC': list(columns_in_ch_not_in_ipc) + [None]*(len(columns_in_ipc_not_in_ch) - len(columns_in_ch_not_in_ipc)),
    'In_IPC_not_in_CH': list(columns_in_ipc_not_in_ch) + [None]*(len(columns_in_ch_not_in_ipc) - len(columns_in_ipc_not_in_ch))
})

{'AEZ_42000', 'popdensity'}
{'AEZ_31000', 'AEZ_12000', 'AEZ_4000', 'AEZ_40000', 'AEZ_20000', 'AEZ_19000', 'AEZ_30000', 'AEZ_33000', 'AEZ_9000', 'AEZ_17000', 'AEZ_25000', 'AEZ_28000', 'AEZ_43000', 'AEZ_10000', 'AEZ_35000'}


In [59]:
merged_ch_df = merged_ch_df.loc[:, ~merged_ch_df.columns.duplicated()]
ipc_Df = ipc_df.loc[:, ~ipc_df.columns.duplicated()]

In [60]:
#reindex ipc_df and merged_ch_df to have the same columns order
ipc_df = ipc_df.reindex(sorted(ipc_df.columns), axis=1)
merged_ch_df = merged_ch_df.reindex(sorted(merged_ch_df.columns), axis=1)


In [61]:

# append two dataframes, for columns not in one dataframe, fill with NaN
final_df = pd.concat([merged_ch_df, ipc_df], ignore_index=True, sort=False)

In [62]:
# for columns started with AEZ_, fill NaN with 0
aez_columns = [col for col in final_df.columns if col.startswith('AEZ_')]
for col in aez_columns:
    final_df[col] = final_df[col].fillna(0)

In [63]:
#drop duplicates in year month and admin_code
final_df = final_df.drop_duplicates(subset=['year', 'month', 'admin_code'])

In [64]:
#save IPCCH dataset
final_df.to_csv(r"C:\Users\swl00\IFPRI Dropbox\Weilun Shi\Google fund\Analysis\1.Source Data\IPCCH_2017_2025_final_v12102025_with_zscores.csv", index=False)